# EVALUATION

In [1]:
from keras import backend as K
from keras.models import load_model
from keras.optimizers import Adam
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization
from data_generator.object_detection_2d_data_generator import DataGenerator
from eval_utils.average_precision_evaluator import Evaluator

Using TensorFlow backend.


## PASCAL VOC DATA

In [2]:
# Set a few configuration parameters.
img_height = 300
img_width = 300
n_classes = 20
model_mode = 'inference'

## 1. Load trained SSD
You can find the download links to all the trained model weights in the README.

### 1.1. Build the model and load trained weights into it

In [8]:
# 1: Build the Keras model

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, 3),
                n_classes=20,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], # The scales for MS COCO are [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
                two_boxes_for_ar1=True,
                steps=[8, 16, 32, 64, 100, 300],
                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                clip_boxes=False,
                variances=[0.1, 0.1, 0.2, 0.2],
                normalize_coords=True,
                subtract_mean=[123, 117, 104],
                swap_channels=[2, 1, 0],
                confidence_thresh=0.5,
                iou_threshold=0.45,
                top_k=200,
                nms_max_output_size=400)

# 2: Load the trained weights into the model.
weights_path = 'pretrained_weights/VGG_VOC0712_SSD_300x300_iter_120000.h5'

model.load_weights(weights_path, by_name=True)

# 3: Compile the model so that Keras won't complain the next time you load it.
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

W0609 15:41:42.513583  7272 module_wrapper.py:139] From C:\Users\abhinav.jhanwar\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0609 15:41:42.526578  7272 module_wrapper.py:139] From C:\Users\abhinav.jhanwar\Documents\git\object-detection-classification\ssd\keras_loss_function\keras_ssd_loss.py:95: The name tf.log is deprecated. Please use tf.math.log instead.

W0609 15:41:42.549485  7272 deprecation.py:323] From C:\Users\abhinav.jhanwar\Documents\git\object-detection-classification\ssd\keras_loss_function\keras_ssd_loss.py:166: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


Or

### 1.2. Load a trained model

In [ ]:
# TODO: Set the path to the `.h5` file of the model to be loaded.
model_path = 'model.h5'

# We need to create an SSDLoss object in order to pass that to the model loader.
ssd_loss = SSDLoss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

K.clear_session() # Clear previous models from memory.

model = load_model(model_path, custom_objects={'AnchorBoxes': AnchorBoxes,
                                               'L2Normalization': L2Normalization,
                                               'DecodeDetections': DecodeDetections,
                                               'compute_loss': ssd_loss.compute_loss})

## 2. Create a data generator for the evaluation dataset

In [9]:
dataset = DataGenerator()

# TODO: Set the paths to the dataset here.
images_dir = 'pascal voc/test/JPEGImages/'
annotations_dir = 'pascal voc/test/Annotations/'
image_set_filename = 'pascal voc/test/ImageSets/Main/test.txt'

# The XML parser needs to now what object class names to look for and in which order to map them to integers.
classes = ['background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat',
           'chair', 'cow', 'diningtable', 'dog',
           'horse', 'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']

dataset.parse_xml(images_dirs=[images_dir],
                  image_set_filenames=[image_set_filename],
                  annotations_dirs=[annotations_dir],
                  classes=classes,
                  include_classes='all',
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False)

Processing image set 'test.txt': 100%|████████████████████████████████████████████| 4952/4952 [00:26<00:00, 190.09it/s]


## 3. Run the evaluation
The evaluator roughly performs the following steps: It runs predictions over the entire given dataset, then it matches these predictions to the ground truth boxes, then it computes the precision-recall curves for each class, then it samples 11 equidistant points from these precision-recall curves to compute the average precision for each class, and finally it computes the mean average precision over all classes.

In [10]:
evaluator = Evaluator(model=model,
                      n_classes=n_classes,
                      data_generator=dataset,
                      model_mode=model_mode)

results = evaluator(img_height=img_height,
                    img_width=img_width,
                    batch_size=8,
                    data_generator_mode='resize',
                    round_confidences=False,
                    matching_iou_threshold=0.5,
                    border_pixels='include',
                    sorting_algorithm='quicksort',
                    average_precision_mode='sample',
                    num_recall_points=11,
                    ignore_neutral_boxes=True,
                    return_precisions=True,
                    return_recalls=True,
                    return_average_precisions=True,
                    verbose=True)

mean_average_precision, average_precisions, precisions, recalls = results

Number of images in the evaluation dataset: 4952

Producing predictions batch-wise: 100%|██████████████████████████████████████████████| 619/619 [34:24<00:00,  3.33s/it]
Matching predictions to ground truth, class 1/20.: 100%|██████████████████████████| 240/240 [00:00<00:00, 10022.83it/s]
Matching predictions to ground truth, class 2/20.: 100%|██████████████████████████| 295/295 [00:00<00:00, 10564.46it/s]
Matching predictions to ground truth, class 3/20.: 100%|███████████████████████████| 358/358 [00:00<00:00, 8965.35it/s]
Matching predictions to ground truth, class 4/20.: 100%|██████████████████████████| 229/229 [00:00<00:00, 10933.73it/s]
Matching predictions to ground truth, class 5/20.: 100%|██████████████████████████| 203/203 [00:00<00:00, 11289.06it/s]
Matching predictions to ground truth, class 6/20.: 100%|██████████████████████████| 203/203 [00:00<00:00, 11288.91it/s]
Matching predictions to ground truth, class 7/20.: 100%|████████████████████████| 1135/1135 [00:00<00:00, 1250

## 4. Visualize the results

In [11]:
for i in range(1, len(average_precisions)):
    print("{:<14}{:<6}{}".format(classes[i], 'AP', round(average_precisions[i], 3)))
print()
print("{:<14}{:<6}{}".format('','mAP', round(mean_average_precision, 3)))

aeroplane     AP    0.71
bicycle       AP    0.721
bird          AP    0.62
boat          AP    0.595
bottle        AP    0.339
bus           AP    0.8
car           AP    0.8
cat           AP    0.809
chair         AP    0.485
cow           AP    0.698
diningtable   AP    0.681
dog           AP    0.798
horse         AP    0.809
motorbike     AP    0.72
person        AP    0.622
pottedplant   AP    0.341
sheep         AP    0.625
sofa          AP    0.755
train         AP    0.809
tvmonitor     AP    0.694

              mAP   0.672
